In [118]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [119]:
fake = faker.Faker(locale='ru')

In [120]:
yandex_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'content-type': 'application/json',
    'referer': 'http://yandex.ru/jobs/vacancies'
}

In [188]:
URL = "http://yandex.ru/jobs/api/publications/?page_size=1000&text="
VACANCY_INFO_URL = "http://yandex.ru/jobs/vacancies/"

In [184]:
# proxy = {
#     'http': 'http://130.61.236.9:80'
# }

In [189]:
clear_string = lambda x: re.sub(' +', ' ', x.replace('\n', '\n ')).strip()
title_to_url = lambda x: "-".join(re.sub('\s+', ' ', re.sub(r'[^\w\s-]', '', x.lower())).split(' '))

In [192]:
result = requests.get(URL, headers=yandex_headers)
result

<Response [200]>

In [193]:
all_jobs = result.json()['results']
len(all_jobs)

774

In [201]:
datetime.now()

datetime.datetime(2022, 6, 17, 5, 25, 27, 724828)

In [194]:
jobs_info = []
for job in all_jobs:
    url = VACANCY_INFO_URL + title_to_url(job['title']) + '-' + str(job['id'])
    internal_id = job['id']
    summary = job['short_summary']
    jobs_info.append((url, internal_id, job['title']))

In [204]:
yandex_jobs = []
for job in tqdm(jobs_info):
    result = requests.get(job[0], headers=yandex_headers)

    if result.status_code != 200:
        print(f'Non 200 for {job[0]}. skipping...')
        continue

    soup = BeautifulSoup(result.text)

    description_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'})
    description = clear_string(description_div.text) if description_div is not None else None

    responsibilities_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'})
    responsibilities = clear_string(responsibilities_div.text) if responsibilities_div is not None else None

    qualifications_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'})
    qualifications = clear_string(qualifications_div.text) if qualifications_div is not None else None

    if description is None and responsibilities is None and qualifications is None:
        print(f'All None for {job[0]}. skipping...')
        continue

    job_dict = {
        'title': job[2],
        'internal_id': job[1],
        'url': job[0],
        'description': description,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'Yandex',
        'publish_date': datetime.now()
    }
    yandex_jobs.append(job_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 774/774 [08:55<00:00,  1.45it/s]


In [205]:
snapshot = pd.DataFrame(yandex_jobs)
print(len(snapshot))
snapshot.sample(5)

774


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
743,Разработчик Logos,5940,http://yandex.ru/jobs/vacancies/разработчик-lo...,Группа Logos входит в службу инфраструктуры ре...,Что нужно делать:\n строить общую DMP-платформ...,"Мы ждем, что вы:\n хорошо знаете Python;\n уме...",Yandex,2022-06-17 05:42:21.628440
380,Разработчик на Go или Python в Биллинг,9093,http://yandex.ru/jobs/vacancies/разработчик-на...,"Биллинг — внутренний сервис Яндекса, обслужива...","Что нужно делать:\n разбираться в том, как раб...","Мы ждем, что вы:\n работали с базами данных (P...",Yandex,2022-06-17 05:38:08.515235
404,Региональный представитель в геосервисы,9071,http://yandex.ru/jobs/vacancies/региональный-п...,Геосервисы Яндекса помогают нашим клиентам раз...,Что нужно делать:\n заниматься активными прода...,"Мы ждем, что вы:\n имеете опыт работы в холодн...",Yandex,2022-06-17 05:38:25.064374
394,Руководитель аналитических проектов,9083,http://yandex.ru/jobs/vacancies/руководитель-а...,"Crowd — это внутренний сервис, который решает ...",Что нужно делать:\n организовать работу аналит...,"Мы ждем, что вы:\n несколько лет управляли ком...",Yandex,2022-06-17 05:38:17.857127
698,Разработчик интерфейсов в Почту,6546,http://yandex.ru/jobs/vacancies/разработчик-ин...,"Яндекс 360 — виртуальное рабочее пространство,...",Что нужно делать:\n проектировать архитектуру ...,"Мы ждем, что вы:\n пишете на JavaScript;\n зна...",Yandex,2022-06-17 05:41:50.400335


In [208]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774 entries, 0 to 773
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             774 non-null    object        
 1   internal_id       774 non-null    int64         
 2   url               774 non-null    object        
 3   description       732 non-null    object        
 4   responsibilities  720 non-null    object        
 5   qualifications    774 non-null    object        
 6   company           774 non-null    object        
 7   publish_date      774 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 48.5+ KB


In [209]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'17-06-2022'

In [210]:
snapshot.to_csv(f'../data/yandex/{current_date}.csv')

In [195]:
result = requests.get(jobs_info[19][0], headers=yandex_headers)
result

<Response [200]>

In [196]:
result.request.url

'https://yandex.ru/jobs/vacancies/python-%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA-%D0%B2-wiki-4005'

In [197]:
soup = BeautifulSoup(result.text)

In [198]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'}).text)

'Наша команда делает сервисы, которые формируют базу знаний Яндекса. Ежедневно сотрудники Яндекса обращаются за рабочей документацией, совместно редактируют статьи, делятся рабочими материалами, готовят отчеты. Работоспособность всех этих процессов обеспечивается нашими решениями.\n Исторически большинство крупных проектов написано на Python3 (Django, FastApi).'

In [199]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'}).text)

'Что нужно делать:\n повышать удобство работы с сервисами;\n улучшать быстродействие;\n проектировать и внедрять функциональность в ответ на новые потребности пользователей и бизнеса.'

In [200]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'}).text)

'Мы ждем, что вы:\n имеете опыт разработки (production-ready) и поддержки веб-сервисов. При этом важен не столько сам язык, сколько умение мыслить, писать эффективный код и обладать собственным видением;\n любите сложные и крупные сервисы;\n готовы к тесному взаимодействию с потребителями.'